
# Cosmos‑Predict **2.5** — Installation via **uv** (Python 3.10)

This notebook automates the **official** installation steps from the Cosmos‑Predict 2.5 *Setup Guide* using the **uv** package manager. It creates an isolated `.venv` with **Python 3.10**, installs the package, and registers a Jupyter kernel so you can use it interactively.

**System Requirements (from the guide):**
- NVIDIA GPU with **Ampere** (RTX 30 series, **A100**) or newer
- NVIDIA **driver ≥ 570.124.06** (compatible with **CUDA 12.8.1**)
- **Linux x86‑64**, **glibc ≥ 2.31** (e.g., Ubuntu ≥ 22.04)
- **Python 3.10**

> Checkpoints are fetched automatically during inference and post‑training. To change where they’re cached, set the `HF_HOME` environment variable.


## 1) Environment Checks

In [ ]:

import os, re, sys, platform, subprocess, textwrap

def run(cmd):
    print(f"$ {cmd}")
    p = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if p.stdout: print(p.stdout.strip())
    if p.stderr and p.returncode != 0: print(p.stderr.strip())
    return p

print("🔎 Host")
print("OS:", platform.platform())
print("Python:", sys.version.split()[0])
try:
    print("glibc:", platform.libc_ver())
except Exception as e:
    print("glibc: <unavailable>", e)

print("\n🔎 NVIDIA Driver / CUDA")
out = run("nvidia-smi --query-gpu=name,driver_version,compute_cap --format=csv,noheader 2>/dev/null")
if out.returncode != 0:
    # Fallback to generic nvidia-smi
    out2 = run("nvidia-smi 2>/dev/null")
    text = (out.stdout or "") + "\n" + (out2.stdout or out2.stderr or "")
else:
    text = out.stdout

m_drv = re.search(r"([0-9]+\.[0-9]+\.[0-9]+)", text) or re.search(r"Driver Version:\s*([0-9.]+)", text)
driver = m_drv.group(1) if m_drv else "unknown"
m_cuda = re.search(r"CUDA Version:\s*([0-9.]+)", text)
cuda = m_cuda.group(1) if m_cuda else "unknown"

print("Driver:", driver)
print("CUDA:  ", cuda)

def vtuple(v):
    try: return tuple(map(int, v.split(".")))
    except: return (0,0,0)

meets_driver = vtuple(driver) >= (570,124,6)
print("Meets driver ≥ 570.124.06:", meets_driver)

print("\n⚠️  Note: Cosmos‑Predict 2.5 requires Ampere or newer (compute capability ≥ 8.0). "
      "Some cloud notebooks may assign older GPUs.")


## 2) Install **uv** (package manager)

In [ ]:

%%bash
set -euo pipefail
echo "Installing uv…"
curl -LsSf https://astral.sh/uv/install.sh | sh
~/.local/bin/uv --version || true

# Persist PATH for current shell sessions launched by bash in this notebook
echo 'export PATH="$HOME/.local/bin:$PATH"' >> ~/.bashrc
export PATH="$HOME/.local/bin:$PATH"


## 3) Clone **cosmos-predict2.5**

In [ ]:

%%bash
set -euo pipefail
REPO_URL="https://github.com/nvidia-cosmos/cosmos-predict2.5.git"
if [ -d "cosmos-predict2.5" ]; then
  echo "Repo already exists, pulling latest…"
  (cd cosmos-predict2.5 && git pull --ff-only)
else
  echo "Cloning $REPO_URL"
  git clone "$REPO_URL"
fi


## 4) Create `.venv` (Python 3.10) with **uv** and install the package

In [ ]:

%%bash
set -euo pipefail
cd cosmos-predict2.5

# Create/refresh the project virtual environment from pyproject/uv.lock
uv --version
echo "Syncing dependencies into .venv (this installs Python 3.10 as needed)…"
uv sync

echo "Registering Jupyter kernel: Python 3.10 (cosmos-predict2.5)"
uv run python -m ipykernel install --user --name cosmos-predict2_5 --display-name "Python 3.10 (cosmos-predict2.5)"

echo "Verifying Python inside .venv:"
uv run python -c "import sys; print(sys.version); print(sys.executable)"



> **Alternative (conda/active env)**  
> If you already activated a Python 3.10 environment (e.g., conda), you can install into the **active** env instead of creating `.venv`:
>
> ```bash
> cd cosmos-predict2.5
> uv sync --active --inexact
> ```
> This follows the official guide’s alternative path.


## 5) (Optional) Configure `HF_HOME` for checkpoint cache

In [ ]:

import os, pathlib, subprocess, textwrap

# Choose a cache directory (edit as desired)
cache_dir = "/content/hf_cache" if Path("/content").exists() else str(Path.home() / "hf_cache")
Path(cache_dir).mkdir(parents=True, exist_ok=True)

# Persist for future shells
bashrc_line = f'export HF_HOME="{cache_dir}"\n'
with open(Path.home()/".bashrc", "a") as f:
    f.write(bashrc_line)

os.environ["HF_HOME"] = cache_dir
print("HF_HOME set to:", os.environ["HF_HOME"])


## 6) Smoke Test (import the package)

In [ ]:

%%bash
set -euo pipefail
cd cosmos-predict2.5
echo "Trying to import cosmos_predict2 from the .venv…"
uv run python - << 'PY'
import sys
print("Python:", sys.version)
try:
    import cosmos_predict2
    ver = getattr(cosmos_predict2, "__version__", "unknown")
    print("✅ cosmos_predict2 import OK | version:", ver)
except Exception as e:
    print("❌ Import failed:", e)
    raise SystemExit(1)
PY



## 7) Notes & Troubleshooting

- **Automatic checkpoints**: Models are downloaded on first use. To relocate the cache, set `HF_HOME` (see the cell above).
- **Driver/CUDA errors**: If you see *“CUDA driver version insufficient”*, update your NVIDIA driver to a version compatible with **CUDA 12.8.1**.
  Check with:
  ```bash
  nvidia-smi | grep "CUDA Version:"
  ```
- **OOM**: Prefer **2B** models, reduce resolution/FPS/batch size, or use multi‑GPU.
- **Docker path** (advanced): If you have Docker + NVIDIA Container Toolkit installed, you can run the project containerized:
  ```bash
  docker run --gpus all --rm -v .:/workspace -v /workspace/.venv -it $(docker build -q .)
  ```
